In [ ]:
dbd_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
f1_nat, f2_nat = compute_weighted_frequencies(dbd_msa, 22, 0.2);
@load "../data_Genie/pars_dbd.jld2"


T_quench = 0.8; Ts = [T_quench + 0.5*i  for i in 1:6]; start_msa = []; N_seq = 2000;

for temp in Ts
    start_msa0 = dbd_msa[:,rand(1:size(dbd_msa,2), N_seq)]; 
    @time res = run_evolution(start_msa0, h_dbd, J_dbd, p = 0.5, temp = temp, N_points = 5, N_steps = 5*10^6);
    println(temp)
    push!(start_msa, res.step_msa[end])
end


N_steps = 1*10^7; N_Ts = length(Ts); NN_points = 80; steps = unique([
        trunc(Int,10^y) for y in range(log10(1), log10(N_steps), 
                length=NN_points)]); N_points = length(steps); hams = zeros(N_Ts, N_points); var_hams = zeros(N_Ts, 
    N_points); ens = zeros(N_Ts, N_points); var_ens = zeros(N_Ts, 
    N_points); min_ens = zeros(N_Ts, N_points); cor1 = zeros(N_Ts, 
    N_points); cor2 = zeros(N_Ts, N_points); pair_hams = zeros(N_Ts, N_points);res_all = []; 

for i in 1:length(Ts)
    res = run_evolution(start_msa[i], h_dbd, J_dbd, p = 0.5, temp = T_quench, N_points = NN_points, N_steps = N_steps);
    println(i)
    push!(res_all, res)  
end


res_eq = run_evolution(dbd_msa[:,rand(1:24000,2000)], h_dbd, J_dbd, p = 0.5, temp = T_quench, N_points = NN_points, N_steps = N_steps);
f1_eq, f2_eq = compute_weighted_frequencies(res_eq.step_msa[end], 22, 0.);

i = 0
for res in res_all
    i+=1
    for n in 1:N_points
        hams[i,n] = mean(ham_dist(start_msa[i], res.step_msa[n]))
        var_hams[i,n] = var(ham_dist(start_msa[i], res.step_msa[n]))
        pair_hams[i,n] = pairwise_ham_dist(res.step_msa[n], n_seq = N_seq)
        en = energy(res.step_msa[n], h_dbd, J_dbd)
        min_ens[i,n] = minimum(en)
        ens[i,n] = mean(en)
        var_ens[i,n] = var(en)
        f1,f2 = compute_weighted_frequencies(res.step_msa[n], 22, 0.)
        cor1[i,n] = cor(f1[:], f1_eq[:])
        cor2[i,n] = cor(f2[:], f2_eq[:])
    end
end


i = 0
for x in res_all
    i+=1
    for n in 1:N_points
        f1,f2 = compute_weighted_frequencies(x.step_msa[n], 22, 0.)
        cor1[i,n] = cor(f1[:], f1_eq[:])
        cor2[i,n] = cor(f2[:], f2_eq[:])
    end
end


steps = res_all[1].steps;z_min = minimum(Ts); z_max = maximum(Ts);normalized_Ts = [
    (z - z_min) / (z_max - z_min) for z in Ts]; cmap = get_cmap("viridis"); colors = [cmap(z) for z in 
        normalized_Ts];

close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, cor1[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Pearson Correlation")
plt.title("1-point correlation")
plt.savefig("../cor_1_point_new.png")

close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, cor2[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Pearson Correlation")
plt.title("2-point correlation")
plt.savefig("../cor_2_point_new.png")




close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, hams[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Mean Hamming Distance")
plt.savefig("../mean_ham_dist.png")

close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, var_hams[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Var Hamming Distance")
plt.savefig("../var_ham_dist.png")


close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, pair_hams[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Pairwise Hamming Distance")
plt.savefig("../pair_ham_dist.png")


close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, cor1[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end


close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, ens[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

#plt.ylim(20, 50)
plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Mean Energy")
plt.savefig("../mean_energy.png")

close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, var_ens[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Energy Variance")
plt.savefig("../var_energy.png")


close("all")
plt.plot()
for i in 1:N_Ts
    plt.plot(steps, min_ens[i,:], color=colors[i], label = "T = $(round(Ts[i], digits = 2))")
end

plt.legend()
plt.xlabel("MCMC steps")
plt.xscale("log")
plt.ylabel("Minimum Energy")
plt.savefig("../min_energy.png")

    
#### other part #####


ens = zeros(N_Ts, N_points); var_ens = zeros(N_Ts, N_points); 

for i in 1:length(Ts)
    for n in 1:N_points
        en = energy(res_all[i].step_msa[n], h_dbd, J_dbd)
        ens[i,n] = mean(en)
        var_ens[i,n] = var(en)
    end     
end    
    

In [ ]:
for i in 1:length(Ts)
    println(i)
    for n in 1:N_points
        pair_hams[i,n] = pairwise_ham_dist(res_all[i].step_msa[n], n_seq = 100)
        #=hams[i,n] = mean(ham_dist(start_msa[i], res_all[i].step_msa[n]))
        var_hams[i,n] = var(ham_dist(start_msa[i], res_all[i].step_msa[n]))
        en = energy(res_all[i].step_msa[n], h_dbd, J_dbd)
        min_ens[i,n] = minimum(en)
        ens[i,n] = mean(en)
        var_ens[i,n] = var(en)
        f1,f2 = compute_weighted_frequencies(res_all[i].step_msa[n], 22, 0.2)
        cor1[i,n] = cor(f1[:], f1_nat[:])
        cor2[i,n] = cor(f2[:], f2_nat[:])=#
    end     
end